# Mean Variance Frontier w/w.o Short Sales Constraints

This code below calculates mean variances frontiers (a) when there are no restrictions on the portfolio weights and (b) when we impose the restriction that no weights can be negative.

The latter case requires a numerical minimization routine (the code uses quadprog), which comes from the packages MathProgBase and Ipopt. You may have to install these to run the code (see below for instructions).

## Load Packages

In [1]:
using MathProgBase, Ipopt 

include("printmat.jl")    

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# The Inputs to the MV Calculations

In [3]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;          #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printlnPs(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030


## From Ch2 Notebook (MV Frontier without portfolio restrictions)

In [4]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [5]:
mustar = linspace(0.04,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRp[i] = MVCalc(mustar[i],μ,Σ)[1]
end    

plot(StdRp*100,mustar*100,color=:red,linewidth=2,legend=false)
plot!(xlims=(0,15),ylims=(0,15))
title!("Mean variance frontier (no portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean variance frontier (no portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 451.533,277.329 444.481,274.79 437.497,272.25 430.585,269.711 423.75,267.171 416.994,264.632 410.324,262.093 403.743,259.553 397.258,257.014 390.873,254.474 
 384.594,251.935 378.428,249.395 372.381,246.856 366.461,244.316 360.673,241.777 355.027,239.237 349.529,236.698 344.19,234.158 339.017,231.619 334.019,229.079 
 329.207,226.54 324.591,224 320.18,221.461 315.985,218.921 312.017,216.382 308.285,213.842 304.802,211.303 301.576,208.763 298.62,206.224 295.941,203.684 
 293.551,201.145 291.456,198.606 289.667,196.066 288.188,193.527 287.027,190.987 286.188,188.448 285.675,185.908 285.49,183.369 285.633,180.829 286.104,178.29 
 286.901,175.75 288.021,173.211 289.459,170.671 291.209,168.132 293.264,165.592 295.617,163.053 298.259,160.513 301.181,157.974 304.372,155.434 307.823,152.895 
 311.523,150.355 315.462,147.816 319.628,145.276 324.012,142.737 328.602,140.198 333.39,137.658 338.364,135.119 343.515,132.579 348.833,130.04 354.311,127.5 
 359.939,124.961 365.708,122.421 371.613,119.882 377.644,117.342 383.795,114.803 390.059,112.263 396.431,109.724 402.904,107.184 409.472,104.645 416.131,102.105 
 422.876,99.5658 429.702,97.0264 436.604,94.4869 443.579,91.9474 450.622,89.4079 457.731,86.8684 464.901,84.329 472.13,81.7895 479.414,79.25 486.751,76.7105 
 494.139,74.171 501.573,71.6316 509.053,69.0921 516.577,66.5526 524.141,64.0131 531.744,61.4736 539.384,58.9342 547.06,56.3947 554.77,53.8552 562.512,51.3157 
 570.285,48.7762 578.087,46.2368 585.918,43.6973 593.776,41.1578 601.659,38.6183 609.567,36.0788 617.498,33.5394 625.453,30.9999 633.429,28.4604 641.426,25.9209 
 649.443,23.3815 
 "/>

## MV Frontier when Short Sales are Not Allowed

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).

In [6]:
function MeanVarNoSSPs(mu,Sigma,mustar)
    K = length(mustar)                #MV with no short-sales, numerical minimization
    N = length(mu)
    vv = find((mustar .>= minimum(mu)) & (mustar .<= maximum(mu)))  #solve only if feasible
    lb = zeros(N,1)              #w >= 0
    ub = ones(N,1)               #w <= 1
    A  = [ones(1,N);mu']         #1'w=1, mu'w = mustar
    w_p   = fill(NaN,(K,N))
    StdRp = fill(NaN,K)                       #to store results in
    for i = vv
        b  = [1;mustar[i]]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        w_i = Sol.sol
        if Sol.status == :Optimal
          w_p[i,:] = w_i'
          StdRp[i] = sqrt((w_i'*Sigma*w_i)[1])
        end
    end
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [7]:
Std_no_ss, = MeanVarNoSSPs(μ,Σ,mustar)  

plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlims=(0,15),ylims=(0,15))
title!("Mean variance frontier (no portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean variance frontier (no portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 451.533,277.329 444.481,274.79 437.497,272.25 430.585,269.711 423.75,267.171 416.994,264.632 410.324,262.093 403.743,259.553 397.258,257.014 390.873,254.474 
 384.594,251.935 378.428,249.395 372.381,246.856 366.461,244.316 360.673,241.777 355.027,239.237 349.529,236.698 344.19,234.158 339.017,231.619 334.019,229.079 
 329.207,226.54 324.591,224 320.18,221.461 315.985,218.921 312.017,216.382 308.285,213.842 304.802,211.303 301.576,208.763 298.62,206.224 295.941,203.684 
 293.551,201.145 291.456,198.606 289.667,196.066 288.188,193.527 287.027,190.987 286.188,188.448 285.675,185.908 285.49,183.369 285.633,180.829 286.104,178.29 
 286.901,175.75 288.021,173.211 289.459,170.671 291.209,168.132 293.264,165.592 295.617,163.053 298.259,160.513 301.181,157.974 304.372,155.434 307.823,152.895 
 311.523,150.355 315.462,147.816 319.628,145.276 324.012,142.737 328.602,140.198 333.39,137.658 338.364,135.119 343.515,132.579 348.833,130.04 354.311,127.5 
 359.939,124.961 365.708,122.421 371.613,119.882 377.644,117.342 383.795,114.803 390.059,112.263 396.431,109.724 402.904,107.184 409.472,104.645 416.131,102.105 
 422.876,99.5658 429.702,97.0264 436.604,94.4869 443.579,91.9474 450.622,89.4079 457.731,86.8684 464.901,84.329 472.13,81.7895 479.414,79.25 486.751,76.7105 
 494.139,74.171 501.573,71.6316 509.053,69.0921 516.577,66.5526 524.141,64.0131 531.744,61.4736 539.384,58.9342 547.06,56.3947 554.77,53.8552 562.512,51.3157 
 570.285,48.7762 578.087,46.2368 585.918,43.6973 593.776,41.1578 601.659,38.6183 609.567,36.0788 617.498,33.5394 625.453,30.9999 633.429,28.4604 641.426,25.9209 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008100; stroke-width:1.6; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 406.802,229.079 396.234,226.54 385.935,224 375.93,221.461 366.248,218.921 356.921,216.382 347.981,213.842 339.466,211.303 331.414,208.763 323.868,206.224 
 316.87,203.684 310.467,201.145 304.703,198.606 299.624,196.066 295.273,193.527 291.69,190.987 288.911,188.448 286.964,185.908 285.87,183.369 285.643,180.829 
 286.104,178.29 286.901,175.75 288.021,173.211 289.459,170.671 291.209,168.132 293.264,165.592 295.617,163.053 298.259,160.513 301.181,157.974 304.372,155.434 
 307.823,152.895 311.523,150.355 315.462,147.816 319.628,145.276 324.012,142.737 328.602,140.198 333.39,137.658 338.551,135.119 345.51,132.579 354.445,130.04 
 365.183,127.5 377.545,124.961 391.352,122.421 406.434,119.882 422.634,117.342 439.812,114.803 457.841,112.263 476.613,109.724 496.033,107.184 516.019,104.645 
 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)